# Часть 1. Pandas

У вас есть данные mobile_app_events.csv об использовании мобильного приложения пользователями, зарегистрированными с 29 июля по 1 сентября 2019 года:
- user_id - индентификатор пользователя;
- event_date - время события;
- event_type - тип события:
  - registration - регистрация в приложении; 
  - simple_event - событие "клик"-а приложении; 
  - purchase - оплата в приложении; 
  - purchase_amount - размер оплаты

In [1]:
import numpy as np
import pandas as pd

In [2]:
mobile_app_events = pd.read_csv('mobile_app_events.csv')

In [3]:
mobile_app_events.head(3)

,user_id,event_date,event_type,purchase_amount
0,c40e6a,2019-07-29 00:02:15,registration,NaN
1,a2b682,2019-07-29 00:04:46,registration,NaN
2,9ac888,2019-07-29 00:13:22,registration,NaN


In [4]:
mobile_app_events.describe(include='all')

,user_id,event_date,event_type,purchase_amount
count,79742,79742,79742,6207.000000
unique,9996,78509,3,NaN
top,aef0ea,2019-08-29 17:13:38,simple_event,NaN
freq,24,3,63539,NaN
mean,NaN,NaN,NaN,30.035444
std,NaN,NaN,NaN,14.101013
min,NaN,NaN,NaN,10.000000
25%,NaN,NaN,NaN,20.000000
50%,NaN,NaN,NaN,30.000000
75%,NaN,NaN,NaN,40.000000


#### 1. Разделить пользователей на когорты на основе номера недели регистрации пользователя в приложении — в качестве идентификатора когорты использовать номер недели в году.

In [5]:
mobile_app_events.loc[:, 'event_date'] = pd.to_datetime(mobile_app_events['event_date'], 
                                                        infer_datetime_format=True)

In [6]:
mobile_app_events['event_type'].unique()

array(['registration', 'simple_event', 'purchase'], dtype=object)

In [7]:
user_cohort_count = mobile_app_events.query('event_type == "registration"')[['user_id', 'event_date']
                                                                           ].groupby('user_id').count()

In [8]:
user_cohort_count[user_cohort_count.event_date >1]

,event_date
user_id,


In [9]:
user_cohort = mobile_app_events.query('event_type == "registration"')[['user_id', 'event_date']]

In [10]:
user_cohort['cohort'] = user_cohort['event_date'].dt.isocalendar().week

In [11]:
user_cohort.head()

,user_id,event_date,cohort
0,c40e6a,2019-07-29 00:02:15,31
1,a2b682,2019-07-29 00:04:46,31
2,9ac888,2019-07-29 00:13:22,31
3,93ff22,2019-07-29 00:16:47,31
4,65ef85,2019-07-29 00:19:23,31


#### 2. Сколько уникальных пользователей в когорте с ID 33?

In [12]:
len(user_cohort[user_cohort.cohort == 33])

2045

#### 3. Для каждого события посчитать столбец lifetime - кол-во недель с момента регистрации до этого события. Например, если пользователь из когорты с ID 31 совершил действие 3го августа, то соответствующее значение будет 0. Если же тот же пользователь совершит событие 5го августа, то значение будет 1.

In [13]:
mobile_app_events['reg_date'] = mobile_app_events.user_id.map(dict(zip(user_cohort.user_id, 
                                                                       user_cohort.event_date)))

In [14]:
mobile_app_events['reg_cohort'] = mobile_app_events.user_id.map(dict(zip(user_cohort.user_id, 
                                                                       user_cohort.cohort)))

In [15]:
mobile_app_events['event_cohort'] = mobile_app_events['event_date'].dt.isocalendar().week

In [16]:
mobile_app_events['lifetime'] = mobile_app_events['event_cohort'] - mobile_app_events['reg_cohort']

#### 4. Рассчитать таблицу изменений Retention Rate (Коэффициент удержания клиентов = Количество клиентов в конце периода - Количество клиентов, приобретенных в течение периода / Количество клиентов в начале периода * 100%) для всех когорт и значений lifetime

In [17]:
mobile_app_events.head()

,user_id,event_date,event_type,purchase_amount,reg_date,reg_cohort,event_cohort,lifetime
0,c40e6a,2019-07-29 00:02:15,registration,NaN,2019-07-29 00:02:15,31,31,0
1,a2b682,2019-07-29 00:04:46,registration,NaN,2019-07-29 00:04:46,31,31,0
2,9ac888,2019-07-29 00:13:22,registration,NaN,2019-07-29 00:13:22,31,31,0
3,93ff22,2019-07-29 00:16:47,registration,NaN,2019-07-29 00:16:47,31,31,0
4,65ef85,2019-07-29 00:19:23,registration,NaN,2019-07-29 00:19:23,31,31,0


In [18]:
mobile_app_events_groupped = mobile_app_events.groupby(['user_id', 'lifetime']).first().reset_index()

In [19]:
mobile_app_events_groupped.head()

,user_id,lifetime,event_date,event_type,purchase_amount,reg_date,reg_cohort,event_cohort
0,00049f,0,2019-08-18 16:21:02,registration,10.0,2019-08-18 16:21:02,33,33
1,00049f,1,2019-08-19 22:12:46,simple_event,NaN,2019-08-18 16:21:02,33,34
2,00049f,2,2019-08-27 12:37:53,simple_event,NaN,2019-08-18 16:21:02,33,35
3,0005f9,0,2019-08-29 16:24:42,registration,NaN,2019-08-29 16:24:42,35,35
4,00082c,0,2019-08-03 17:37:37,registration,NaN,2019-08-03 17:37:37,31,31


In [20]:
mobile_app_events_groupped.reg_cohort.unique()

array([33, 35, 31, 34, 32], dtype=int32)

In [21]:
abs_retention = mobile_app_events_groupped[['reg_cohort', 'lifetime', 'user_id']
                          ].groupby(['reg_cohort', 'lifetime']
                                   ).count().rename(columns={'user_id':'users'}).unstack(
                                        ).fillna(0)

In [22]:
RR = abs_retention.apply(lambda x: round(100 * x / x[("users", 0)], 2), axis=1)

In [23]:
RR.style.background_gradient(cmap='Blues', axis=1)

#### 5. Рассчитать трёхнедельный Retention Rate для когорты с ID 32. Ответ дать числом с плавающей точкой, округлённым до 2 знаков после запятой.

In [24]:
RR.loc[32, ("users", 3)]

36.12

#### 6. Рассчитать таблицу изменений ARPPU (Average Revenue Per Paying User) для всех когорт и значений lifetime.

In [25]:
paying_users_df = mobile_app_events[~pd.isnull(mobile_app_events.purchase_amount)]

In [26]:
# check the minimum purchase amount

paying_users_df.purchase_amount.min()

10.0

In [27]:
ARPPU = paying_users_df[['reg_cohort', 'lifetime', 'purchase_amount', 'user_id']
                       ].groupby(['reg_cohort', 'lifetime']).agg(
        revenue = pd.NamedAgg(column='purchase_amount', aggfunc='sum'),
        n_users = pd.NamedAgg(column='user_id', aggfunc='count'))

ARPPU['ARPPU'] = round(ARPPU['revenue'] / ARPPU['n_users'], 2)
ARPPU = ARPPU[['ARPPU']].unstack().fillna(0)

In [28]:
ARPPU.style.background_gradient(cmap='Blues', axis=1)

#### 7. Рассчитать трёхнедельный ARPPU для когорты с ID 31. Ответ дать числом с плавающей точкой, округлённым до 2 знаков после запятой.

In [29]:
ARPPU.loc[31, ("ARPPU", 3)]

29.14

#### 8. Рассчитать медианное время между регистрацией и первой покупкой. Ответ дать целым числом в секундах.

In [30]:
paying_users_df.event_type.unique()

array(['purchase'], dtype=object)

In [31]:
first_purchases = paying_users_df.sort_values('event_date').groupby('user_id').first()

In [32]:
first_purchases['time_delta'] = (first_purchases['event_date'] - first_purchases['reg_date']
                                                                            ) // np.timedelta64(1, 's')

In [33]:
first_purchases['time_delta'].median()

434774.0

# Часть 2. Функции

#### Задача 7.
Написать функцию *is_prime(number)*, принимающую 1 аргумент — число, и возвращающую *True*, если оно простое, и *False* — иначе. Пользоваться циклом нельзя.

In [34]:
def is_prime(number: int) -> bool:
    """
    Returns True, if the number is prime, else False.
    """
    if number < 1:
        return False
    def check_div(n, d):
        if d == 1:
            return True
        elif n % d == 0:
            return False
        else:
            return check_div(n, d - 1)
    d = int(number ** 0.5)
    if check_div(number, d):
        return True
    else:
        return False

In [35]:
is_prime(5)

True

In [36]:
is_prime(100)

False

#### Задача 8.
В разных папках лежат разные файлы. Нужно из всех папок извлечь эти файлы и
пронумеровать их по порядку (*либо по порядку извлечения, либо в алфавитном порядке)* и распечатать построчно в формате: номер, название файла. Требование — программа работает для любой степени вложенности папок.
Написать функцию *file_numerator(path)*, которая на вход принимает строку — путь до директории, ничего не возвращает, печатает в output строки: “номер, название файла” (без кавычек)

In [37]:
import os

def file_numerator(path: str) -> None:
    """
    Prints the name and serial number of files in the path directory.
    """
    n = 0
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            n += 1
            print(n, name)

#### Задача 9.
Написать рекурсивную функцию *palindrome(string)*, определяющую, является входная строка палиндромом (читается с обоих концов одинаково).

In [38]:
def palindrome(string: str) -> bool:
    """
    Returns True, if the string is palindrome, else False.
    """
    if len(string) < 2:
        return True
    if len(string) >= 2:
        if string[0] == string[-1]:
            return palindrome(string[1:-1])
        else:
            return False

In [39]:
palindrome('1aabaab')

False

In [40]:
palindrome('aaaa')

True

#### Задача 10.
Написать функцию *merge_sort(arr)*, выполняющую сортировку слиянием с помощью рекурсии.

In [41]:
from typing import List

In [42]:
def merge_sort(arr: List[int]) -> List[int]:
    """
    Returns the array sorted with merge sort algorithm.
    """
    if len(arr) < 2:
        return arr
    mid_index = len(arr) // 2
    left, right = arr[:mid_index], arr[mid_index:]
    left = merge_sort(left)
    right = merge_sort(right)
    def merge_two_arrays(array1: List[int], array2: List[int]) -> List[int]:
        """
        Returns the merged from array1 and array2 sorted array.
        """
        ret_list = []
        i, j = 0, 0
        while i < len(array1) and j < len(array2):
            if array1[i] < array2[j]:
                ret_list.append(array1[i])
                i += 1
            else:
                ret_list.append(array2[j])
                j += 1
        return ret_list + array1[i:] + array2[j:]
    return merge_two_arrays(left, right)

In [43]:
merge_sort([1, 5, 3, 9, 0, 2, 3, 2])

[0, 1, 2, 2, 3, 3, 5, 9]